In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digsite/IMG_0054.MOV


In [ ]:
!git clone https://github.com/dokooh/vggt
%cd vggt
!pip install -e .
!pip install -r requirements_demo.txt

Cloning into 'vggt'...
remote: Enumerating objects: 579, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 579 (delta 7), reused 0 (delta 0), pack-reused 558 (from 1)
Receiving objects: 100% (579/579), 64.72 MiB | 38.02 MiB/s, done.
Resolving deltas: 100% (190/190), done.
/kaggle/working/vggt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 11.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 20.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 100.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [3]:
!git fetch
!git pull 

Already up to date.


**Inference Step**

In [4]:
%ls

CODE_OF_CONDUCT.md  examples/                     requirements.txt
CONTRIBUTING.md     LICENSE.txt                   training/
demo_colmap.py      pyproject.toml                vggt/
demo_gradio.py      README.md                     vggt_refinement_pipeline.py
demo_viser.py       requirements_demo.txt         vggt_video_inference.py
docs/               requirements-postprocess.txt  visual_util.py


In [ ]:
!python vggt_video_inference.py \
    --video /kaggle/input/digsite/IMG_0054.MOV \
    --frame_interval 15 \
    --max_frames 50 \
    --detect_blur \
    --select_frames 30 \
    --selection_method keyframe

EXTRACTING FRAMES FROM VIDEO
Video: /kaggle/input/digsite/IMG_0054.MOV
Output directory: /kaggle/working/IMG_0054_frames

Video Properties:
  Total frames: 308
  FPS: 30.00
  Resolution: 1920x1080
  Duration: 10.27 seconds

Extraction Settings:
  Frame interval: 15
  Skip initial frames: 0
  Max frames: 50
  Motion blur detection: True
308it [00:05, 61.22it/s]                                                        

EXTRACTION COMPLETE
Total frames processed: 308
Frames extracted: 21
Frames skipped (motion blur): 0
Output directory: /kaggle/working/IMG_0054_frames

RUNNING VGGT INFERENCE
Device: cuda
Precision: torch.float16

Loading model: facebook/VGGT-1B
(This will download weights on first run)
config.json: 100%|████████████████████████████| 62.0/62.0 [00:00<00:00, 557kB/s]
model.safetensors: 100%|████████████████████| 5.03G/5.03G [00:29<00:00, 172MB/s]

Processing 21 images...


In [ ]:
%ls /kaggle/working/IMG_0054_frames*

In [ ]:
!zip -r predictions.zip /kaggle/working/predictions/

**Postprocess step**

**Note:** VGGT outputs point clouds in `.npy` or `.pkl` format. The refinement pipeline will automatically detect these formats and convert them to `.ply` for better compatibility with visualization tools.

In [ ]:
# Run in a Kaggle cell
!pip install -r requirements-refine.txt

In [ ]:
!pip install lightglue #--no-deps
#!pip install kornia roma

In [ ]:
# First, convert VGGT output (.npy/.pkl) to PLY format for compatibility
!python vggt_refinement_pipeline.py \
    --scene_dir="/kaggle/working/IMG_0054_frames" \
    --point_cloud="/kaggle/working/predictions" \
    --convert_format ply

# Then optionally run refinement with BA:
# !python vggt_refinement_pipeline.py \
#     --scene_dir="/kaggle/working/IMG_0054_frames" \
#     --point_cloud="/kaggle/working/IMG_0054_frames/refined/pointclouds/converted.ply" \
#     --use_ba \
#     --visualize

In [ ]:
#With GCP scaling:
!python vggt_refinement_pipeline.py \
    --scene_dir=/path/to/scene \
    --point_cloud=/path/to/vggt_output.ply \
    --gcp_file=ground_control_points.json \
    --voxel_size=0.005 \
    --visualize

In [ ]:
#With known distance scaling:
!python vggt_refinement_pipeline.py \
    --scene_dir=/path/to/scene \
    --point_cloud=/path/to/vggt_output.ply \
    --known_distance=2.5 \
    --point_indices 100 500 \
    --visualize